### Блок теоретических вопросов 

*Пусть мы решаем задачу множественной классификации для разметки 4 уникальных лейблов (классов). Выберите верное утверждение про количество базовых моделей в подходах OneVsOne и OneVsRest*: 

1. Для использования OneVsOne придется обучить 4 базовых модели, а для подхода OneVsRest 6.
2. Для использования OneVsOne придется обучить 6 базовых модели, а для подхода OneVsRest 4.
3. Для использования OneVsOne придется обучить 4 базовых модели, а для подхода OneVsRest 4.
4. Для использования OneVsOne придется обучить 6 базовых модели, а для подхода OneVsRest 6.



**Ответ: 2)** Для каждой пары $(i, j)$ из возможных классов в OneVsOne строится по модели бинарной лкассификации. Среди 4 лейблов таких пар найдется ровно 6 уникальных: $(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)$! В случае OneVsRest все проще - посмотрим на количество лейблов!

___________________________________________

*Выберите верные утверждения относительно подходов микро- и макроусреднения*: 

1.	При микро-усреднении вклад каждого класса в итоговую метрику одинаков.
2.	При микро-усреднении вклад каждого класса в итоговую метрику пропорционален размеру данного класса.
3.	При макро-усреднении вклад каждого класса в итоговую метрику одинаков.
4.	При макро-усреднении вклад каждого класса в итоговую метрику пропорционален размеру данного класса.



**Ответ: 2, 3)** Макро-усреднение, в отличие от микро-, усредняет уже нечувствительные к дисбалансу классов метрики. Поэтому вклад и одинаков.

___________________________________________

### Блок практики

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

Мы будем работать с данными агрегатора такси [Sigma Cabs](https://www.kaggle.com/datasets/arashnic/taxi-pricing-with-mobility-analytics). В зависимости от характеристик поездки требуется предсказать один из трех типов повышенного ценообразования: [1, 2, 3]. Таким образом, это поможет компании оптимально мэтчить такси и клиентов. 

In [ ]:
df = pd.read_csv('sigma_cabs.csv')
df.shape

In [ ]:
# Занесем индекс колонку
df = df.set_index('Trip_ID')
df.head()

Описание признаков:

1. **Trip_ID**: ID for TRIP
2. **Trip_Distance**: The distance for the trip requested by the customer
3. **TypeofCab**: Category of the cab requested by the customer
4. **CustomerSinceMonths**: Customer using cab services since n months; 0 month means current month
5. **LifeStyleIndex**: Proprietary index created by Sigma Cabs showing lifestyle of the customer based on their behaviour
6. **ConfidenceLifeStyle_Index**: Category showing confidence on the index mentioned above
7. **Destination_Type**: Sigma Cabs divides any destination in one of the 14 categories.
8. **Customer_Rating**: Average of life time ratings of the customer till date
9. **CancellationLast1Month**: Number of trips cancelled by the customer in last 1 month
10. **Var1**, **Var2** and **Var3**: Continuous variables masked by the company. Can be used for modelling purposes
11. **Gender**: Gender of the customer

**SurgePricingType**: Target (can be of 3 types)


### EDA 
Заполните пропуски в вещественных признаках медианой, а в категориальных - самым популярным классом. Изобразите марицу корреляций и выведите топ5 пар самых коррелированных признаков.

Так как в сумме уникальных значений различных категориальных признаков окажется не супер-много, примените `One-Hot-Encoding` для них. Не забудьте в методе `pd.get_dummies` указать параметр `drop_first=True`.

In [ ]:
# Помотрим на баланс классов
df['Surge_Pricing_Type'].value_counts()

In [ ]:
# Пропущенные переменные в вещественных признаках
num_cols = df.select_dtypes(exclude='object').columns
df[num_cols].isna().sum()

In [ ]:
# Заполним медианой
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [ ]:
plt.title("Correaltion heatmap")
sns.heatmap(df[num_cols].corr(), cmap='inferno');

In [ ]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

get_top_abs_correlations(df[num_cols])

In [ ]:
df.describe(include='object')

In [ ]:
# Пропуски в категориальных заполним самым популярным значением
df = df.fillna(df.mode().iloc[0])
df.isna().sum()

In [ ]:
# Не так много классов, можно юзать OHE
df.select_dtypes('object').nunique()

In [ ]:
# Закодируем категориальные признаки
df = pd.get_dummies(df, drop_first=True)

X = df.drop('Surge_Pricing_Type', axis=1)
y = df['Surge_Pricing_Type']

### Training

In [ ]:
np.random.seed(2022)

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Разобьем на трейн-тест

X_train, X_test, y_train, y_test  = train_test_split(X, y, 
                                                     test_size=0.2, 
                                                     shuffle=True, 
                                                     random_state=2022)

**Задание 1.** Обучите One-vs-Rest Logreg. Посчитайте precision, recall, f1-score и усредните по всем классам с помощью micro, macro и weighted avg. Здесь и далее округляйте до 3 знака после запятой.

Чтобы отдельно и долго не вычислять метрики, можно воспользоваться `classification_report` из `sklearn.metrics`!

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# Обучение
logit = LogisticRegression()
pipe1 = Pipeline([('scaler', StandardScaler()), 
                  ('one_vs_all', OneVsRestClassifier(logit))])

pipe1.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pipe1.predict(X_test), digits=3))

Подберите оптимальные гиперпараметры модели с помощью `GridSearchCV()` из предложенных. Для лучшего набора гиперпараметров посчитайте те же самые метрики. Валидировать параметры необходимо по `accuracy`. В этот раз проведем настояющую процедуру Кросс-Валидации! 

Для этого в метод `fit` передадим тренировочную часть наших данных, в параметр `cv` ничего не будем передавать (по дефолту 5-fold Кросс-Валидация будет проведена), а итоговые метрики замерим на тесте!

In [ ]:
param_grid = {'one_vs_all__estimator__penalty': ['l1', 'l2', 'elasticnet'],
              'one_vs_all__estimator__C': [0.001, 0.01, 0.1, 1]}

In [ ]:
# Подбор параметров
from sklearn.model_selection import GridSearchCV

grid1 = GridSearchCV(pipe1, param_grid, cv=5)
grid1.fit(X_train, y_train)

print(f"Best parameters: {grid1.best_params_}")
print(classification_report(y_test, grid1.predict(X_test), digits=3))

Изобразите три калибровочные кривые для Logistic Classifier: 0-vs-rest, 1-vs-rest, 2-vs-rest. Хорошо ли откалиброван обученный классификатор? 

-- *Кривые достаточно близки к диагонали! Хорошо откалиброван.*

Заметьте, что `predict_proba` возвращает список из вероятностей для всех наших классов!

In [ ]:
grid1.predict_proba(X_test)

In [ ]:
# Закодируем таргет 

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y_test.values.reshape(-1, 1))

In [ ]:
from sklearn.calibration import calibration_curve
plt.figure(figsize=(8, 5))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

# Калибровочные кривые
for label in range(3):
    prob_pos = grid1.predict_proba(X_test)[:, label]
    fraction_of_positives, mean_predicted_value = calibration_curve(y_ohe[:, label],
                                                                    prob_pos,
                                                                    n_bins=15)
    plt.plot(mean_predicted_value, fraction_of_positives, "s-", label=f"{label} vs Rest")

plt.ylabel("Fraction of positives")
plt.xlabel("Mean predicted value")
plt.title('Logistic OVR Calibration curves')
plt.legend();

**Задание 2.** Обучите логистическую регрессию с гиперпараметрами из первого задания на полиномиальных признаках до 4 степени. Сравните метрики с первым заданием.


Пример: Пусть у нас был единственный признак 

$$
d_j = [1, 2, 3, 4]
$$

Тогда полиномиальные признаки до 4 степени от такого будут иметь вид:

$$
d_j^1 = [1, 2, 3, 4]
$$

$$
d_j^2 = [1, 4, 9, 16]
$$

$$
d_j^3 = [1, 8, 27, 64]
$$

$$
d_j^4 = [1, 16, 81, 256]
$$

P.S. Бинарные колонки нет смысла возводить в какие-то степени, поэтому возьмем исключительно вещественные из базовых. 

Для этого можно воспользоваться классическим циклом (или уроком из занятия про `Sberbank Housing Market`).

P.S.S Зачастую еще, создаваю полиномиальные фичи, учитывают "пересечения" признаков, то есть, например, из векторов признаков $d_j, d_i$ генерируют не просто новые степени $d_j^2, d_i^2, d_j^3, d_i^3...$, а еще и признаки вида $d_j \cdot d_i, d_j^2 \cdot d_i, d_j \cdot d_i^2...$, но здесь ограничьтесь просто степенями!

In [ ]:
# Создание полиномиальных признаков
X_polinomial = X.copy()


for col in num_cols.drop('Surge_Pricing_Type'):
    data_part = pd.concat([X[col]**(1+i) for i in range(4)], axis=1)
    data_part.columns = [col + f"_power_{i+1}" for i in range(4)]
    
    X_polinomial = X_polinomial.drop(col, axis=1)
    X_polinomial = pd.concat((X_polinomial, data_part), axis=1)
    
X_polinomial.shape

In [ ]:
X_pol_train, X_pol_test, y_train, y_test  = train_test_split(X_polinomial, y, 
                                                             test_size=0.2, 
                                                             shuffle=True, 
                                                             random_state=2022)

In [ ]:
%%time
logit = LogisticRegression(C=0.001, penalty='l2')
pipe2 = Pipeline([('scaler', StandardScaler()), 
                  ('one_vs_all', OneVsRestClassifier(logit))])

pipe2.fit(X_pol_train, y_train)

print(classification_report(y_test, pipe2.predict(X_pol_test), digits=3))

По аналогии с первым заданием изобразите три калибровочные кривые. Стало ли лучше?

-- *Неоднозначно*

In [ ]:
#Закодируем таргет 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y_test.values.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

# Калибровочные кривые
for label in range(3):
    prob_pos = pipe2.predict_proba(X_pol_test)[:, label]
    fraction_of_positives, mean_predicted_value = calibration_curve(y_ohe[:, label], prob_pos, n_bins=15)
    plt.plot(mean_predicted_value, fraction_of_positives, "s-", label=f"{label} vs Rest")

plt.ylabel("Fraction of positives")
plt.xlabel("Mean predicted value")
plt.title('Logistic OVR Calibration curves')
plt.legend();

**Задание 3.** Обучите на датасете без полиномиальных признаков One-vs-One `SGDClassifier` из `sklearn.linear_model`, который использует стохастический градиентный спуск (узнаете о нем позже) и может обучать как `SVM`, так и, например, `LogReg`, если указать в качестве параметра `loss` либо `hinge`, либо `log` соответственно!

Посчитайте precision, recall, f1-score и усредните по всем классам с помощью micro, macro и weighted avg.

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, 
                                                     test_size=0.2, 
                                                     shuffle=True, 
                                                     random_state=2022)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier

# Обучение
pipe_ovo = Pipeline([('scaler', StandardScaler()), 
                     ('one_vs_one', OneVsOneClassifier(SGDClassifier()))])

pipe_ovo.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test, pipe_ovo.predict(X_test), digits=3))

Подберите оптимальные гиперпараметры модели с помощью `GridSearchCV()`. При этом переберите всевозможные функции потерь. Таким образом, при `loss = 'hinge'`, мы обучим SVM, при `loss = 'log'` мы обучим логистическую регрессию и т.д.

Используйте прием с Кросс-Валидацией при подборе параметров, как ранее, а также замерьте метрики на тесте.

In [ ]:
param_grid = {'one_vs_one__estimator__loss': ['hinge', 'log', 'modified_huber'],
              'one_vs_one__estimator__penalty': ['l1', 'l2'],
              'one_vs_one__estimator__alpha': [0.001, 0.01, 0.1]}

In [ ]:
%%time

grid_ovo = GridSearchCV(pipe_ovo, param_grid, cv=5)
grid_ovo.fit(X_train, y_train)

print(f"Best parameters: {grid_ovo.best_params_}")
print(classification_report(y_test, grid_ovo.predict(X_test), digits=3))

Можно ли однозначной сказать, какой подход оказался лучше: One-vs-Rest или One-vs-One?

-- *Кажется, что у каждого подхода есть свои плюсы и минусы. Хотя в МО обычно опираются на качество работы алгоритмов, и если оно существенное, то можно однозначно сказать, какой из них лучше*